# Logistic Regression to recognize hand-written digits(0 to 9)

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [0]:
data = loadmat('machine-learning-ex3/ex3/ex3data1.mat')
data

{'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [0]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [0]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [0]:
# Cost function for logistic regression which returns J
def cost(theta, X, y, lam):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    # number of training examples
    m = len(y)
    
    term1 = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    term2 = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (lam / (2 * m)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    J = (1 / m) * (np.sum(term1 - term2)) + reg
    
    return J

In [0]:
def gradient(theta, X, y, lam):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    # number of training examples
    m = len(y)
    error = sigmoid(X * theta.T) - y
    
    grad = ((1 / m) * (X.T * error)).T + (lam / m) * theta
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

In [0]:
from scipy.optimize import minimize

# OnevsAll method to train k classifiers for k different classes
def oneVsAll(X, y, k, lam):
    m = X.shape[0]
    n = X.shape[1]
    
    # array to store parameters values for k classifiers
    all_theta = np.zeros((k, n+1))
    
    X = np.insert(X, 0, 1, axis=1)
    
    initial_theta = np.zeros(n+1)
    
    # labels in y are indexed 1 to 9 and digit 0 is mapped to index 10
    for i in range(1, k+1):
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = y_i.reshape(m, 1)
        
        # minimize the cost function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, lam), method='TNC', jac=gradient)
        all_theta[i-1, :] = fmin.x
        
    return all_theta

In [0]:
# checking shape of some arrays
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, 1, axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [0]:
k = len(np.unique(data['y']))
lam = 1

all_theta = oneVsAll(data['X'], data['y'], k, lam)
all_theta

array([[-2.38312114e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30441457e-03, -7.47969349e-10,  0.00000000e+00],
       [-3.18171737e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.46188951e-03, -5.08709849e-04,  0.00000000e+00],
       [-4.79911754e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.86939691e-05, -2.47202954e-07,  0.00000000e+00],
       ...,
       [-7.98655664e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.91603139e-05,  7.20529930e-06,  0.00000000e+00],
       [-4.57447341e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33707881e-03,  9.99155510e-05,  0.00000000e+00],
       [-5.40501584e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16611806e-04,  7.88209230e-06,  0.00000000e+00]])

In [0]:
def predictAll(X, all_theta):
    m = X.shape[0]
    n = X.shape[1]
    k = all_theta.shape[0]
    
    X = np.matrix(np.insert(X, 0, 1, axis=1))
    all_theta = np.matrix(all_theta)
    
    # compute the probability for each class for each training set
    # h -> m x k
    h = sigmoid(X * all_theta.T)
    
    # choose index with maximum probability
    # hmax -> m x 1
    hmax = np.argmax(h, axis=1)
    
    # because our array was zero-indexed we need to add one for the true label prediction
    hmax = hmax + 1
    
    return hmax

In [0]:
predictions = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a,b) in zip(predictions, data['y'])]
accuracy = (sum(correct) / len(correct)) * 100
print('accuracy: %0.2f' % accuracy)

accuracy: 94.46
